In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [2]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,11/16/2013 01:45:00 AM,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,09/28/2016 04:45:00 PM,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,12/10/2013 07:15:00 AM,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,02/27/2014 09:00:00 AM,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,10/31/2016 04:15:00 PM,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [3]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,2013-11-16T01:45:00,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,2016-09-28T04:45:00,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,2013-12-10T07:15:00,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,2014-02-27T09:00:00,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,2016-10-31T04:15:00,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [4]:
# Save config information
url = "https://api.darksky.net/forecast/"
exception_count = 0

weather_df=pd.DataFrame()

# calculate begining of iteration from last line in csv file if existant
try:
    with open("Output/weather.csv", "r") as f:
        a = int(pd.read_csv("Output/weather.csv", header=None).tail(1)[0]+1)
except:
    a = 0

n = a
        
# Build query URL and DataFrame
for i in range(a, len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info and save to DataFrame
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    
    except:
        if i-n <= 1: exception_count += 1 # count consecutive failed calls
        else: exception_count = 0
            
        n = i
        
        if exception_count >= 5: 
            print(f"Request no {i} failed, 5 consecutive requests have failed, ending itteration")
            break
        else: 
            print(f"Request no {i} failed, skipping to next line")
weather_df

Request no 6097 failed, skipping to next line
Request no 6626 failed, skipping to next line
Request no 6729 failed, skipping to next line
Request no 6767 failed, skipping to next line
Request no 6880 failed, skipping to next line
Request no 6881 failed, skipping to next line
Request no 6882 failed, skipping to next line
Request no 6883 failed, skipping to next line
Request no 6884 failed, skipping to next line
Request no 6885 failed, 5 consecutive requests have failed, ending itteration


,Trip ID,time,summary,precipIntensity,precipProbability,temperature,apparentTemperature,windSpeed,icon
5876,bce8287c97fa9d4e9f1c978e515c06f113b2baf0,2013-03-15T07:30:00,Partly Cloudy,0.0000,0.00,32.19,32.19,1.75,partly-cloudy-day
5877,bbfc9d2f809ec527ef30d199ca294b826c161024,2013-03-31T03:45:00,Overcast,0.0000,0.00,45.54,43.80,3.90,cloudy
5878,b074c7cb47610a5164d4033c61ddd3f2b46d3a1b,2014-07-25T09:45:00,Mostly Cloudy,0.0000,0.00,67.77,67.77,3.31,partly-cloudy-day
5879,be1776856ab07984e9b3d89f190aabee73628c79,2015-07-23T06:15:00,Clear,0.0000,0.00,64.66,64.66,0.05,clear-day
5880,bc101b87683dcc7ffd951b13922b9cef7ed7774a,2013-01-13T04:30:00,Overcast,0.0000,0.00,29.78,29.78,1.62,cloudy
5881,bfcb43f6a88b2583c6235dc1f506dd772c31eeee,2016-05-21T03:45:00,Clear,0.0000,0.00,54.41,54.41,1.47,clear-night
5882,b33d0752a00164a4d6ee4742f46de56ffa29810e,2014-05-17T10:00:00,Partly Cloudy,0.0000,0.00,53.17,53.17,3.73,partly-cloudy-day
5883,ba7b7d9061aacdff3075da19fff655b9afadce0b,2013-12-20T10:30:00,Light Rain,0.0150,0.90,34.81,30.89,4.43,rain
5884,b33fb0dd58f7bb25935d289e0cf83516ac0ffa99,2013-08-28T05:45:00,Mostly Cloudy,0.0000,0.00,75.52,76.48,0.52,partly-cloudy-night
5885,c31d9a44085fdfea26fbc68eb70bed4eb2188993,2013-09-14T12:45:00,Partly Cloudy,0.0000,0.00,67.36,67.36,1.86,partly-cloudy-day


In [5]:
# Save results in csv output file (append if file already exists)
try: os.mkdir('Output')
except: pass

try:
    with open("Output/weather.csv", "a") as f:
        weather_df.to_csv(f, header=False)
except:
    weather_df.to_csv("Output/weather.csv")